In [1]:
import efinance as ef

In [2]:
import pandas as pd
import numpy as np
import talib  # 用于计算技术指标
import tkinter as tk
from tkinter import simpledialog
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# 设置Seaborn风格
sns.set(style="whitegrid")

In [3]:
matplotlib.use('TkAgg')  # 使用Tk后端生成独立窗口（确保已安装tkinter）
matplotlib.font_manager.fontManager.addfont('C:/Windows/Fonts/msyh.ttc')  # 微软雅黑路径
plt.rcParams['font.family'] = 'Microsoft YaHei'

In [4]:

# 创建一个隐藏的主窗口
root = tk.Tk()
root.withdraw()

# 弹出输入对话框
user_input = simpledialog.askstring(
    title="股票代码输入",
    prompt="请输入股票代码:",
    initialvalue='000001'  # 默认值
)

# 处理用户点击取消或关闭窗口的情况
stock_code = user_input if user_input else '000001'

# 后续代码...
print("当前股票代码:", stock_code)

当前股票代码: ZH


In [5]:
import datetime

# 创建一个隐藏的主窗口
root = tk.Tk()
root.withdraw()

# 弹出输入对话框
user_input = simpledialog.askstring(
    title="回测天数输入",
    prompt="请输入回测天数:(0=实时策略)",
    initialvalue='0'  # 默认值
)

# 处理用户点击取消或关闭窗口的情况
stock_day = user_input if user_input else '0'

# 获取当前日期
end_date = datetime.datetime.now()

# 计算n天前的日期
end_date = end_date - datetime.timedelta(days=int(stock_day))

# 将日期格式化为字符串（假设API需要字符串格式的日期）
#start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y%m%d')

print("当前回测天数:", stock_day)

当前回测天数: 0


In [6]:
def market_analysis(data):
    # 判断当前趋势
    if data['MACD'].iloc[-1] > data['MACD_signal'].iloc[-1]:
        trend = "上升"
    elif data['MACD'].iloc[-1] < data['MACD_signal'].iloc[-1]:
        trend = "下降"
    else:
        trend = "盘整"

    # 判断市场强度
    if data['RSI'].iloc[-1] > 70:
        strength = "强"
    elif data['RSI'].iloc[-1] < 30:
        strength = "弱"
    else:
        strength = "中"

    # 判断爆发力
    if data['Volume_Ratio'].iloc[-1] > 1.5:
        momentum = "强"
    elif data['Volume_Ratio'].iloc[-1] < 0.5:
        momentum = "弱"
    else:
        momentum = "中"

    # 建议最大杠杆
    if strength == "强" and momentum == "强":
        leverage = 3.0
    elif strength == "中" and momentum == "中":
        leverage = 2.0
    else:
        leverage = 1.0

    return {
        "当前趋势": trend,
        "市场强度": strength,
        "爆发力": momentum,
        "当日建议最大杠杆": leverage
    }


In [7]:
def trading_plan(data):
    close_price = data['收盘'].iloc[-1]
    macd_hist = data['MACD_hist'].iloc[-1]
    macd_signal = data['MACD_signal'].iloc[-1]
    rsi = data['RSI'].iloc[-1]

    # 判断交易方向
    if (macd_hist > 0 and rsi < 70):
        direction = "做多"
    elif macd_hist < 0 and rsi > 30:
        direction = "做空"
    else:
        direction = "观望"

    # 建仓价格
    entry_price = close_price

    # 建仓仓位
    if direction == "做多":
        position = 0.5  # 50%资金比例
    elif direction == "做空":
        position = 0.3  # 30%资金比例
    else:
        position = 0.0

    # 杠杆倍数
    leverage = market_status["当日建议最大杠杆"]

    # 止损和止盈
    stop_loss = entry_price * 0.95  # 5%止损
    take_profit = entry_price * 1.10  # 10%止盈

    return {
        "交易方向": direction,
        "建仓价格": round(entry_price, 2),
        "建仓仓位": round(position * 100, 2),
        "杠杆倍数": leverage,
        "首次止损": round(stop_loss, 2),
        "首次止盈": round(take_profit, 2)
    }


In [8]:
def batch_entry_plan(data):
    close_price = data['收盘'].iloc[-1]

    # 第一批建仓
    first_entry_price = close_price
    first_position = 0.3  # 30%资金比例
    first_leverage = market_status["当日建议最大杠杆"]

    # 第二批建仓
    second_entry_price = close_price * 0.98  # 2%回调
    second_position = 0.2  # 20%资金比例
    second_leverage = market_status["当日建议最大杠杆"]

    return {
        "第一批 (开盘时机)": {
            "触发价格": round(first_entry_price, 2),
            "仓位比例": round(first_position * 100, 2),
            "杠杆倍数": first_leverage
        },
        "第二批 (盘中时机)": {
            "触发价格": round(second_entry_price, 2),
            "仓位比例": round(second_position * 100, 2),
            "杠杆倍数": second_leverage
        }
    }



In [9]:
def profit_taking_plan(data):
    close_price = data['收盘'].iloc[-1]

    # 盈利加仓点位1
    profit_level_1 = close_price * 1.05  # 5%涨幅
    add_position_1 = 0.2  # 20%加仓比例
    new_stop_loss_1 = close_price * 0.97  # 3%新止损

    # 盈利加仓点位2
    profit_level_2 = close_price * 1.10  # 10%涨幅
    add_position_2 = 0.1  # 10%加仓比例
    new_stop_loss_2 = close_price * 0.95  # 5%新止损

    return {
        "盈利加仓点位1": {
            "触发条件": round((profit_level_1 - close_price) / close_price * 100, 2),
            "加仓比例": round(add_position_1 * 100, 2),
            "新止损位": round(new_stop_loss_1, 2)
        },
        "盈利加仓点位2": {
            "触发条件": round((profit_level_2 - close_price) / close_price * 100, 2),
            "加仓比例": round(add_position_2 * 100, 2),
            "新止损位": round(new_stop_loss_2, 2)
        }
    }



In [10]:
def risk_management_plan():
    return {
        "单笔交易止损比例": 5.00,  # 5%止损
        "日内最大亏损限制": 10.00,  # 10%日内最大亏损
        "强制平仓条件": "日内亏损达到10%或单笔亏损达到5%",
        "当日最大交易次数": 5  # 当日最多交易5次
    }



In [11]:
# 替换为具体股票代码
#stock_code = '000001'

# 获取最新市场数据
latest_data = ef.stock.get_quote_history(stock_code, '19000101', end_date_str)
latest_data = pd.DataFrame(latest_data)

# 计算技术指标
latest_data['MACD'], latest_data['MACD_signal'], latest_data['MACD_hist'] = talib.MACD(latest_data['收盘'], fastperiod=12, slowperiod=26, signalperiod=9)
latest_data['RSI'] = talib.RSI(latest_data['收盘'], timeperiod=14)
latest_data['K'], latest_data['D'] = talib.STOCH(latest_data['最高'], latest_data['最低'], latest_data['收盘'], fastk_period=14, slowk_period=3, slowd_period=3)
latest_data['J'] = 3 * latest_data['K'] - 2 * latest_data['D']
latest_data['MA5'] = talib.MA(latest_data['收盘'], timeperiod=5)
latest_data['MA10'] = talib.MA(latest_data['收盘'], timeperiod=10)
latest_data['Volume_MA5'] = talib.MA(latest_data['成交量'], timeperiod=5)
latest_data['Volume_MA10'] = talib.MA(latest_data['成交量'], timeperiod=10)
latest_data['Volume_Ratio'] = latest_data['成交量'] / latest_data['Volume_MA5']


market_status = market_analysis(latest_data)
print("市场数据日期：", end_date_str)
print("市场状态研判:", market_status)


trading_plan_output = trading_plan(latest_data)
print("交易执行方案:", trading_plan_output)

batch_entry_output = batch_entry_plan(latest_data)
print("分批建仓计划:", batch_entry_output)

profit_taking_output = profit_taking_plan(latest_data)
print("止盈加仓计划:", profit_taking_output)

risk_management_output = risk_management_plan()

#latest_data

市场数据日期： 20250218
市场状态研判: {'当前趋势': '上升', '市场强度': '强', '爆发力': '强', '当日建议最大杠杆': 3.0}
交易执行方案: {'交易方向': '观望', '建仓价格': 5.32, '建仓仓位': 0.0, '杠杆倍数': 3.0, '首次止损': 5.05, '首次止盈': 5.85}
分批建仓计划: {'第一批 (开盘时机)': {'触发价格': 5.32, '仓位比例': 30.0, '杠杆倍数': 3.0}, '第二批 (盘中时机)': {'触发价格': 5.21, '仓位比例': 20.0, '杠杆倍数': 3.0}}
止盈加仓计划: {'盈利加仓点位1': {'触发条件': 5.0, '加仓比例': 20.0, '新止损位': 5.16}, '盈利加仓点位2': {'触发条件': 10.0, '加仓比例': 10.0, '新止损位': 5.05}}


In [12]:
# 按日期筛选数据
latest_data = latest_data[latest_data['日期'] >= '2024-01-01']
#latest_data

In [13]:
#from datetime import date
#today = date.today()
#target_date = date(2024, 11, 7)
#delta = today - target_date
#print(f"相差 {delta.days} 天")

In [14]:
# 绘制价格和技术指标
plt.figure(figsize=(14, 12))

# 子图1: 价格和移动平均线
plt.subplot(3, 1, 1)
plt.plot(latest_data['日期'], latest_data['收盘'], label='收盘价', color='blue')
plt.plot(latest_data['日期'], latest_data['MA5'], label='5日均线', color='orange')
plt.plot(latest_data['日期'], latest_data['MA10'], label='10日均线', color='green')

# 标注建仓价格
entry_price = trading_plan_output["建仓价格"]
plt.axhline(entry_price, color='red', linestyle='--', label='建仓价格')
plt.text(latest_data['日期'].iloc[-1], entry_price, f'建仓价格: {entry_price}', color='red', verticalalignment='bottom')

# 标注止损和止盈
stop_loss = trading_plan_output["首次止损"]
take_profit = trading_plan_output["首次止盈"]
plt.axhline(stop_loss, color='gray', linestyle='--', label='止损')
plt.axhline(take_profit, color='green', linestyle='--', label='止盈')
plt.text(latest_data['日期'].iloc[-1], stop_loss, f'止损: {stop_loss}', color='gray', verticalalignment='bottom')
plt.text(latest_data['日期'].iloc[-1], take_profit, f'止盈: {take_profit}', color='green', verticalalignment='bottom')

plt.title(f'{stock_code} {end_date_str}价格与移动平均线')
plt.legend()
plt.grid(True)
plt.xticks([])

# 子图2: MACD
plt.subplot(3, 1, 2)
plt.plot(latest_data['日期'], latest_data['MACD'], label='MACD', color='blue')
plt.plot(latest_data['日期'], latest_data['MACD_signal'], label='MACD信号线', color='red')
plt.bar(latest_data['日期'], latest_data['MACD_hist'], label='MACD柱状图', color='gray')


# 标注MACD交易信号
action = trading_plan_output["交易方向"]+"/"+market_status["当前趋势"]+"/市场"+market_status["市场强度"]+"/爆发"+market_status["爆发力"]
if action == "做多/上升/市场中/爆发强":
    plt.text(latest_data['日期'].iloc[-1], latest_data['MACD'].iloc[-1], action, color='green', verticalalignment='bottom')
elif action in {"做空/下降/市场中/爆发强", "做空/下降/市场中/爆发中"}:
    plt.text(latest_data['日期'].iloc[-1], latest_data['MACD'].iloc[-1], action, color='red', verticalalignment='bottom')
else:
    plt.text(latest_data['日期'].iloc[-1], latest_data['MACD'].iloc[-1], action, color='grey', verticalalignment='bottom')
#if trading_plan_output["交易方向"] == "做多":
#    plt.text(latest_data['日期'].iloc[-1], latest_data['MACD'].iloc[-1], '做多信号', color='green', verticalalignment='bottom')
#elif trading_plan_output["交易方向"] == "做空":
#    plt.text(latest_data['日期'].iloc[-1], latest_data['MACD'].iloc[-1], '做空信号', color='red', verticalalignment='bottom')

plt.title('MACD指标')
plt.legend()
plt.grid(True)
plt.xticks([])

# 子图3: RSI
plt.subplot(3, 1, 3)
plt.plot(latest_data['日期'], latest_data['RSI'], label='RSI', color='purple')
plt.axhline(70, color='red', linestyle='--', label='超买线 (70)')
plt.axhline(30, color='green', linestyle='--', label='超卖线 (30)')

# 标注RSI超买超卖信号
if latest_data['RSI'].iloc[-1] > 70:
    RSIdata= '超买'+str(round(latest_data['RSI'].iloc[-1],0))
    plt.text(latest_data['日期'].iloc[-1], latest_data['RSI'].iloc[-1], RSIdata, color='red', verticalalignment='bottom')
elif latest_data['RSI'].iloc[-1] < 30:
    RSIdata= '超卖'+str(round(latest_data['RSI'].iloc[-1],0))
    plt.text(latest_data['日期'].iloc[-1], latest_data['RSI'].iloc[-1], RSIdata, color='green', verticalalignment='bottom')

plt.title('RSI指标')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.xticks([])
plt.show(block=False)

C:\Users\wangx\anaconda3\Lib\site-packages\matplotlib\text.py:897: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x = float(self.convert_xunits(self._x))
